In [1]:
from dash import Dash, dcc, html, Input, Output, callback
from dash.dash_table import DataTable
import plotly.express as px

import pandas as pd
import numpy as np
import plotly.express as px
import random
import string
import os


In [2]:
np.random.seed(0)
df = pd.DataFrame({"Col-" + str(i + 1): np.random.rand(100) for i in range(2)})

name = [''.join(random.choices(string.ascii_uppercase +
                             string.digits, k=7)) for i in range(100)]
df['name'] = name

In [3]:
colors = px.colors.diverging.RdYlGn
colors.reverse()
color_scale = [(np.around(i/11,2),np.around((i/11)+(1/11),2),col) for i,col in enumerate(colors)]

In [4]:
colors

['rgb(0,104,55)',
 'rgb(26,152,80)',
 'rgb(102,189,99)',
 'rgb(166,217,106)',
 'rgb(217,239,139)',
 'rgb(255,255,191)',
 'rgb(254,224,139)',
 'rgb(253,174,97)',
 'rgb(244,109,67)',
 'rgb(215,48,39)',
 'rgb(165,0,38)']

In [5]:
color_scale

[(0.0, 0.09, 'rgb(0,104,55)'),
 (0.09, 0.18, 'rgb(26,152,80)'),
 (0.18, 0.27, 'rgb(102,189,99)'),
 (0.27, 0.36, 'rgb(166,217,106)'),
 (0.36, 0.45, 'rgb(217,239,139)'),
 (0.45, 0.55, 'rgb(255,255,191)'),
 (0.55, 0.64, 'rgb(254,224,139)'),
 (0.64, 0.73, 'rgb(253,174,97)'),
 (0.73, 0.82, 'rgb(244,109,67)'),
 (0.82, 0.91, 'rgb(215,48,39)'),
 (0.91, 1.0, 'rgb(165,0,38)')]

In [6]:
col_name = ['Col-1','Col-2']
k = [(col,bin) for bin in color_scale for col in col_name]

style_cond = [{
        'if': {
            'filter_query': '{} > {} && {} <= {}'.format('{'+col_id+'}', schema[0],'{'+col_id+'}', schema[1]),
            'column_id': col_id
        },
        'backgroundColor': schema[2],
        'color': 'black'
    } for col_id,schema in k
]

In [7]:
fig = px.scatter(df,
                 x = "Col-1",
                 y = "Col-2",
                 hover_data = ['name'],
                 height = 800)

In [10]:
def export_selection(exp_df):
    if os.path.isfile('export_data.csv'):
        exp_df.to_csv('export_data.csv', mode='a',header=False, index = False)
    else:
        exp_df.to_csv('export_data.csv', index = False)

    return
    
app = Dash(__name__)
app.layout = html.Div(children =
    [
        html.Div(
            dcc.Graph(id="g1",figure = fig),
            style={'display':'inline-block','vertical-align': 'top',
                   'width':'50%' }),
        html.Div([  
                DataTable(
                    id = 't1',
                    columns = [{'name': i, 'id': i} for i in df.columns],
                    style_data_conditional = style_cond)
                    #ata=df.to_dict('records'))
                ],style={'display':'inline-block','vertical-align': 'top','width' : '40%','height':400})
        
    ]
)

@callback(
    Output("t1", "data"),
    Input("g1", "selectedData")
   )
def select_points(selection):

    temp_df=pd.DataFrame(columns = df.columns)
    if selection is not None:

        point_idxs = [p["pointIndex"] for p in selection["points"]]
        temp_df = df.iloc[point_idxs,::]
        #print([p["customdata"] for p in selection["points"]])
        export_selection(temp_df.copy())
        
    data = temp_df.to_dict('records')
    
    return data

if __name__ == "__main__":
    app.run(debug=True, port = 8051)